In [1]:
import numpy as np
import matplotlib.pyplot as plt
import PIL

%matplotlib qt5

import ipywidgets

In [ ]:
!pip install ipywidgets

!jupyter nbextension enable --py widgetsnbextension

In [2]:
_2pi = np.pi*2.0
_pi = np.pi
_pi2 = np.pi/2.0

### Open images

In [7]:
Ir = [None]*5
Ir[0] = PIL.Image.open('words_5_1.png')
Ir[1] = PIL.Image.open('words_5_2.png')
Ir[2] = PIL.Image.open('words_5_3.png')
Ir[3] = PIL.Image.open('words_5_4.png')
Ir[4] = PIL.Image.open('words_5_5.png')

### Function for interactive selection and segmentation
cutUp, cutDown, cutLeft and cutRight = crop limits

IMG = Image to crop

ICut = reference to result image (list with one image)

In [8]:
def showImage(cutUp = 10, cutDown=-10, cutLeft = 10, cutRight=-10, IMG = None, ICut = []):
    plt.figure(figsize=(14,14))
    
    plt.subplot(1,2,1)
    plt.cla()
    plt.imshow(IMG)
    plt.plot([cutLeft, cutLeft], [0, IMG.shape[0]], linewidth=3)
    plt.plot([IMG.shape[1] + cutRight, IMG.shape[1] + cutRight], [0, IMG.shape[0]], linewidth=3)

    plt.plot([0, IMG.shape[1]], [cutUp, cutUp], linewidth=3)
    plt.plot([0, IMG.shape[1]], [IMG.shape[0] + cutDown, IMG.shape[0] + cutDown], linewidth=3)
    
    ICut[0] = IMG[cutUp:cutDown, cutLeft:cutRight, :]
    
    plt.subplot(1,2,2)
    plt.cla()
    plt.imshow(ICut[0], extent=(0,1,0,1), aspect=1)
    #plt.axis([cutLeft, IMG.shape[1] + cutRight, cutUp, IMG.shape[0] + cutDown], extent=[0,100,0,1], aspect='auto')

Segmentation of image from camera 1

In [38]:
IMGS = [None]*5
ICuts = [None]*5

for i in range(5):
    IMGS[i] = np.array(Ir[i])[:,:,:3]
    ICuts[i] = [None]
    v = ipywidgets.interact(showImage, cutUp = ipywidgets.widgets.IntSlider(min=0, max=IMGS[i].shape[0], step=1, value=10, layout=ipywidgets.Layout(width='100%'), continuous_update=False), 
                         cutDown = ipywidgets.widgets.IntSlider(min=-IMGS[i].shape[1], max=-1, step=1, value=-10, layout=ipywidgets.Layout(width='100%'), continuous_update=False), 
                         cutLeft = ipywidgets.widgets.IntSlider(min=0, max=IMGS[i].shape[1], step=1, value=419, layout=ipywidgets.Layout(width='100%'), continuous_update=False), 
                         cutRight = ipywidgets.widgets.IntSlider(min=-IMGS[i].shape[1], max=-1, step=1, value=-419, layout=ipywidgets.Layout(width='100%'), continuous_update=False), IMG = ipywidgets.fixed(IMGS[i]), ICut = ipywidgets.fixed(ICuts[i]) )



interactive(children=(IntSlider(value=10, continuous_update=False, description='cutUp', layout=Layout(width='1…

interactive(children=(IntSlider(value=10, continuous_update=False, description='cutUp', layout=Layout(width='1…

interactive(children=(IntSlider(value=10, continuous_update=False, description='cutUp', layout=Layout(width='1…

interactive(children=(IntSlider(value=10, continuous_update=False, description='cutUp', layout=Layout(width='1…

interactive(children=(IntSlider(value=10, continuous_update=False, description='cutUp', layout=Layout(width='1…

Segmentation of image from camera 2

## Projection

### geometry

The angles and segments used in the code follows the following diagram:

<img src="draw1.png" width=600>


The panorama image will be formed by uniformly sweeping from 0 to $2\,\pi$ the angle $\theta$ in the diagram. For each angle $\theta$, we compute the angle $\xi$ in the camera image and transfer the pixel line from the camera do the panorama (at position $\theta$)




### Parameters 

a1, a2, a3 = camera angles

qsi = $\xi(\theta)$

o = $\theta(\xi)$


Max angles corresponds to angles there the camera angle $\xi$ becomes tangent to the circle

qMax = $\xi_{max}$

oMax = $\theta_{max}$


lmin_o1, lmin_o2, lmin_o3 = limits that maps the extent of each camera image to angles in $\xi$. Can be $2\,\theta_{max}$ around each camera angle (in which case will be overlaps in the panoramas) or just $120^o$ around each camera (which will mapped to the center of each camera image with no overlap)

In [14]:
R = 11.25
D = 74.5

a = np.linspace(-_pi+_pi/5.0,_pi-_pi/5.0, 5)

qsi = lambda o : np.arctan(  R*np.sin(o)/(D-R*np.cos(o))   )
o = lambda qsi : np.arcsin((D/R)*np.sin(qsi)) - qsi

qMax = np.arcsin(R/D)
oMax = o(qMax)


# lim_o1 = (a1 - oMax, a1 + oMax)
# lim_o2 = (a2 - oMax, a2 + oMax)
# lim_o3 = (a3 - oMax, a3 + oMax)

lim_o = [None]*5
for i in range(5):
    lim_o[i] = (a[i] - _pi/5.0, a[i] + _pi/5.0)

#### Panorama generation function

The function <tt>generatePanorama</tt> generates a panorama W x H big by sweeping $-\pi$ to $\pi$ (mapping to 0 to W in the panorama image) and for each angle (pixel column in the panorama) finds which image contains that pixel column, computes the mapping between $\xi$ and $\theta$, converts (scales) $\theta$ to pixel in the camera and transfer that column of pixels to the panorama.


In [16]:
def isInInterval(o, lims):
    res1 = o >= lims[0] and o <= lims[1]
    res2 = (o + 2.0*np.pi) >= lims[0] and (o + 2.0*np.pi) <= lims[1]
    res3 = (o - 2.0*np.pi) >= lims[0] and (o - 2.0*np.pi) <= lims[1]

    return res1 or res2 or res3

In [59]:
def generatePanorama(W, H, ass, lim_os, Is):

    IPan = np.zeros( (H,W,3), dtype='int' )

    for i in range(W):
        _o = -_pi - i*_2pi/W

        for j in range(5):
            if isInInterval(_o,lim_os[j]):
                _q = qsi(ass[j] - _o)
                p = ( (_q + qMax)/(2.0*qMax) )*Is[j][0].shape[1]

                IPan[:,i,:] = Is[j][0][:H,int(p),:]

    return IPan

W = 1500
H = 512

IPan = generatePanorama(W, H, a, lim_o, ICuts)

            
plt.figure(figsize=(15, 7))
plt.imshow(IPan,extent=[-3,3,-1,1])

#### Interactive visualization of the projections

Interactively plots the spatial position of the light ray from each camera to the center circle, drawing the line corresponding to that ray position in each camera as well as in the panorama.

Notice that the limits of the cameras overlap in the panorama. 

In [58]:
t = np.linspace(0.0,1.0,128).reshape( (1,128) )

Ps = [None]*5
for i in range(5):
    Ps[i] = (D*np.cos(a[i]), D*np.sin(a[i]))

C = np.concatenate( (R*np.cos(_2pi*t), R*np.sin(_2pi*t)) , axis=0 )


def updatePlot(qNorm1 = 0.0, qNorm2 = 0.0, qNorm3 = 0.0, qNorm4 = 0.0, qNorm5 = 0.0):

    qNorms = [qNorm1, qNorm2, qNorm3, qNorm4, qNorm5]
    
    qs = [None]*5
    for i in range(5):
        qs[i] = qMax*qNorms[i]

    os = [None]*5
    for i in range(5):
        os[i] = o(qs[i])
    

    bs = [None]*5
    for i in range(5):
        bs[i] = R*np.sin(os[i] + a[i])

    ds = [None]*5
    for i in range(5):
        ds[i] = R*np.cos(os[i] + a[i])

        
    #plt.figure( figsize=(14, 14) )

    plt.axes([0.0, 0.5, 1.0, 0.5])
    
    plt.plot(C[0,:], C[1,:], color='k')

    for i in range(5):
        plt.plot([Ps[i][0], ds[i]], [Ps[i][1], bs[i]], color='r')
        plt.plot([0, ds[i]], [0, bs[i]], color='r')


    for i in range(5):
        plt.plot([Ps[i][0]], [Ps[i][1]], '.', color='r', markersize=30)

        
    plt.axis('equal')

    for i in range(5):
        plt.axes([0.0 + i/5.0, 0.25, 1/5.0, 0.25])
        plt.imshow(ICuts[i][0], extent=(-1,1,-1,1), aspect=1.0/1.33333)
        plt.plot([qNorms[i], qNorms[i]],[-1, 1], color='r')
    


    plt.axes([0.0, 0.0, 1.0, 0.25])
    plt.imshow(IPan, extent=(-3.14,3.14,-1,1), aspect=0.33333)    
    
    for i in range(5):
        plt.plot([-_pi+(os[i]+a[i])%(_2pi), -_pi+(os[i]+a[i])%(_2pi)],[-1, 1], color='r')
    
    plt.axis([-_pi, _pi, -1, 1])



    
    
qNorms = [None]*5
for i in range(5):
    qNorms[i] = ipywidgets.widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, value=0.0, layout=ipywidgets.Layout(width='100%'), continuous_update=False)
    
ipywidgets.interact(updatePlot, qNorm1 = qNorms[0], qNorm2 = qNorms[1], qNorm3 = qNorms[2], qNorm4 = qNorms[3], qNorm5 = qNorms[4] )  

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='qNorm1', layout=Layout(widt…

<function __main__.updatePlot(qNorm1=0.0, qNorm2=0.0, qNorm3=0.0, qNorm4=0.0, qNorm5=0.0)>